In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [6]:
# MNIST dataset 
train_dataset =  torchvision.datasets.MNIST(root = '../../data',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)
test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                         train = False,
                                         transform=transforms.ToTensor())

# Data Loader 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)


In [8]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i + 1) % 100 == 0:
            print('Epoch [{}/{}], step [{}/{}], Loss: {:.4f}'
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
# Test the model 

# In test phase , we don't need to compute gradients ( for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images: {} %'.format(100*correct / total))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'feedforward_model.ckpt')


    

Epoch [1/5], step [100/600], Loss: 0.4059
Epoch [1/5], step [200/600], Loss: 0.2323
Epoch [1/5], step [300/600], Loss: 0.2910
Epoch [1/5], step [400/600], Loss: 0.0843
Epoch [1/5], step [500/600], Loss: 0.2524
Epoch [1/5], step [600/600], Loss: 0.1464
Epoch [2/5], step [100/600], Loss: 0.1175
Epoch [2/5], step [200/600], Loss: 0.0335
Epoch [2/5], step [300/600], Loss: 0.1203
Epoch [2/5], step [400/600], Loss: 0.1529
Epoch [2/5], step [500/600], Loss: 0.1059
Epoch [2/5], step [600/600], Loss: 0.1007
Epoch [3/5], step [100/600], Loss: 0.0903
Epoch [3/5], step [200/600], Loss: 0.0652
Epoch [3/5], step [300/600], Loss: 0.0543
Epoch [3/5], step [400/600], Loss: 0.1032
Epoch [3/5], step [500/600], Loss: 0.1129
Epoch [3/5], step [600/600], Loss: 0.0542
Epoch [4/5], step [100/600], Loss: 0.0533
Epoch [4/5], step [200/600], Loss: 0.0373
Epoch [4/5], step [300/600], Loss: 0.0327
Epoch [4/5], step [400/600], Loss: 0.0119
Epoch [4/5], step [500/600], Loss: 0.0157
Epoch [4/5], step [600/600], Loss: